<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [2]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])
ratings.drop(['timestamp'],axis=1)
ratings.head(5)

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Genres Matrix

In [3]:
m_cols = ['movie id','movie title','release date','video release date','IMDb URL',
          'unknown','Action','Adventure','Animation','Children','Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
          'Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 100k\\u.item', sep='|', names=m_cols,encoding='latin-1')
movies = movies.set_index('movie id')
movies = movies.drop(['video release date'],axis = 1)
genres = movies.drop(['movie title','release date','IMDb URL'],axis = 1)
genres

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


# Unpivot Ratings Matrix

In [4]:
ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
count = ratings.count(axis=1)
ratings = ratings.fillna(0)
ratings

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Profile matrix

In [5]:
profile = pd.DataFrame((ratings).values.dot(genres.values), columns=genres.columns, index=(ratings).index)
profile

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,
1,4.0,250.0,123.0,40.0,55.0,316.0,86.0,24.0,420.0,7.0,5.0,45.0,38.0,18.0,173.0,172.0,188.0,92.0,22.0
2,0.0,38.0,13.0,4.0,12.0,61.0,34.0,0.0,134.0,3.0,9.0,6.0,3.0,14.0,66.0,15.0,43.0,11.0,0.0
3,0.0,39.0,14.0,0.0,0.0,31.0,30.0,5.0,64.0,0.0,5.0,12.0,4.0,35.0,17.0,22.0,53.0,14.0,0.0
4,0.0,31.0,14.0,0.0,0.0,20.0,19.0,5.0,27.0,0.0,0.0,4.0,5.0,20.0,13.0,23.0,43.0,9.0,0.0
5,4.0,176.0,107.0,53.0,71.0,246.0,35.0,0.0,72.0,5.0,5.0,71.0,40.0,9.0,44.0,116.0,56.0,45.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,73.0,35.0,4.0,4.0,68.0,13.0,0.0,82.0,4.0,0.0,3.0,4.0,4.0,48.0,33.0,49.0,15.0,0.0
940,0.0,79.0,36.0,9.0,21.0,148.0,30.0,0.0,173.0,0.0,12.0,12.0,22.0,13.0,85.0,58.0,67.0,49.0,0.0
941,0.0,38.0,27.0,14.0,9.0,29.0,3.0,0.0,21.0,0.0,0.0,0.0,4.0,5.0,5.0,31.0,28.0,5.0,0.0


# Compute Predictions (Genres . Profile^T)

In [6]:
prediction = pd.DataFrame(genres.values.dot((profile.T).values), columns=(profile.T).columns, index=genres.index)
prediction = prediction.T
prediction

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,411.0,561.0,188.0,986.0,694.0,420.0,592.0,791.0,420.0,512.0,...,438.0,420.0,420.0,420.0,420.0,420.0,361.0,593.0,316.0,420.0
2,77.0,94.0,43.0,233.0,211.0,134.0,149.0,207.0,134.0,145.0,...,81.0,134.0,134.0,134.0,134.0,134.0,109.0,200.0,61.0,134.0
3,31.0,106.0,53.0,134.0,147.0,64.0,86.0,95.0,64.0,78.0,...,92.0,64.0,64.0,64.0,64.0,64.0,70.0,81.0,31.0,64.0
4,20.0,88.0,43.0,78.0,89.0,27.0,50.0,47.0,27.0,36.0,...,74.0,27.0,27.0,27.0,27.0,27.0,56.0,40.0,20.0,27.0
5,370.0,339.0,56.0,494.0,163.0,72.0,188.0,389.0,72.0,117.0,...,232.0,72.0,72.0,72.0,72.0,72.0,100.0,116.0,246.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,76.0,157.0,49.0,223.0,144.0,82.0,115.0,154.0,82.0,97.0,...,122.0,82.0,82.0,82.0,82.0,82.0,97.0,130.0,68.0,82.0
940,178.0,182.0,67.0,400.0,270.0,173.0,231.0,342.0,173.0,222.0,...,146.0,173.0,173.0,173.0,173.0,173.0,152.0,258.0,148.0,173.0
941,52.0,93.0,28.0,88.0,52.0,21.0,52.0,59.0,21.0,26.0,...,66.0,21.0,21.0,21.0,21.0,21.0,33.0,26.0,29.0,21.0


In [7]:
print(prediction.max().max())
print(prediction.min().min())

2333.0
0.0


In [108]:
OldMin = 0
OldMax = 3
NewMin = 1
NewMax = 5
NewValue = (((prediction - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
NewValue

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,2.043511,2.201642,1.636975,2.156914,2.095942,1.603716,2.387595,1.915248,1.603716,2.127274,...,2.002254,1.603716,1.603716,1.603716,1.603716,1.603716,1.866890,1.843372,1.619690,1.603716
2,1.166282,1.190806,1.140998,1.256488,1.331907,1.192650,1.211737,1.235180,1.192650,1.215362,...,1.184352,1.192650,1.192650,1.192650,1.192650,1.192650,1.265944,1.302467,1.131885,1.192650
3,1.037732,1.212158,1.171548,1.156900,1.274946,1.088008,1.175730,1.088544,1.088008,1.165435,...,1.205022,1.088008,1.088008,1.088008,1.088008,1.088008,1.158276,1.099204,1.065354,1.088008
4,1.025033,1.183792,1.153881,1.099309,1.195842,1.036857,1.146414,1.046313,1.036857,1.082313,...,1.173716,1.036857,1.036857,1.036857,1.036857,1.036857,1.136566,1.053818,1.043359,1.036857
5,2.124835,1.752597,1.169344,1.656213,1.288725,1.092093,1.687536,1.608380,1.092093,1.383892,...,1.514292,1.092093,1.092093,1.092093,1.092093,1.092093,1.210813,1.156188,1.486525,1.092093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,1.124553,1.352053,1.173861,1.286325,1.225309,1.116638,1.258929,1.159107,1.116638,1.206090,...,1.292572,1.116638,1.116638,1.116638,1.116638,1.116638,1.249331,1.208868,1.139393,1.116638
940,1.334990,1.369308,1.216954,1.442374,1.403753,1.237773,1.485093,1.384119,1.237773,1.547569,...,1.319476,1.237773,1.237773,1.237773,1.237773,1.237773,1.349332,1.364053,1.293589,1.237773
941,1.199454,1.207788,1.090567,1.121835,1.082776,1.031618,1.194769,1.079101,1.031618,1.052050,...,1.152511,1.031618,1.031618,1.031618,1.031618,1.031618,1.072575,1.030893,1.054290,1.031618


In [110]:
NewValue.min().min()

1.0